# PUMP catalog

TODO:
- [ ] Add MITgcm
- [ ] Display LES too
- [ ] Obs catalog

In [ ]:
%load_ext watermark

from pathlib import Path

from ecgtools import Builder

%watermark -iv

json: 2.0.9



In [ ]:
ROOT = Path("/glade/campaign/cgd/oce/projects/pump/cesm/")

In [ ]:
# MOM6 run catalog
catalog = {
    "baseline": (
        "baseline",
        "gmom.e23.GJRAv3.TL319_t061_zstar_N65.baseline.001.mixpods",
    ),
    "epbl": ("ePBL", "gmom.e23.GJRAv3.TL319_t061_zstar_N65.baseline.epbl.001.mixpods"),
    "kpp.lmd.002": (
        "KPP Ri0=0.5",
        "gmom.e23.GJRAv3.TL319_t061_zstar_N65.baseline.kpp.lmd.002.mixpods",
    ),
    "kpp.lmd.003": (
        "KPP Ri0=0.5, Ric=0.2,",
        "gmom.e23.GJRAv3.TL319_t061_zstar_N65.baseline.kpp.lmd.003.mixpods",
    ),
    "kpp.lmd.004": (
        "KPP ν0=2.5, Ric=0.2, Ri0=0.5",
        "gmom.e23.GJRAv3.TL319_t061_zstar_N65.baseline.kpp.lmd.004.mixpods",
    ),
    "baseline.N150": (
        "baseline N=150",
        "gmom.e23.GJRAv3.TL319_t061_zstar_N150.baseline.mixpods",
    ),
    "kpp.lmd.004.N150": (
        "KPP ν0=2.5, Ric=0.2, Ri0=0.5, N=150",
        "gmom.e23.GJRAv3.TL319_t061_zstar_N150.kpp.lmd.004.mixpods",
    ),
    "new_baseline.hb": (
        "KD=0, KV=0",
        "gmom.e23.GJRAv3.TL319_t061_zstar_N65.new_baseline.hb",
    ),
    "new_baseline.kpp.lmd.004": (
        "KPP ν0=2.5, Ric=0.2, Ri0=0.5",
        "gmom.e23.GJRAv3.TL319_t061_zstar_N65.new_baseline.kpp.lmd.004",
    ),
    "new_baseline.kpp.lmd.005": (
        "KPP ν0=2.5, Ri0=0.5",
        "gmom.e23.GJRAv3.TL319_t061_zstar_N65.new_baseline.kpp.lmd.005",
    ),
}

In [ ]:
simnames = [v[-1] for v in catalog.values()]
path = list((ROOT / simnames[-3] / "run" / "jsons").glob("*.json"))[-1]

In [ ]:
def parse_cesm_mom6_kerchunk_json(file, storage_options=None):
    """Parser for CESM timeseries files"""
    import warnings
    from pathlib import Path

    import fsspec

    import xarray as xr

    if storage_options is None:
        storage_options = {}

    info = dict()
    path = Path(file)
    fs = fsspec.filesystem("reference", fo=file, **storage_options)
    mapper = fs.get_mapper("")
    ds = xr.open_zarr(
        mapper,
        use_cftime=True,
        consolidated=False,
        chunks={},
    )

    if not ds:
        warnings.warn(RuntimeWarning, f"bad file: {file}")
        ds.close()
        return

    info["casename"] = path.parts[-4]
    info["stream"] = path.stem
    info["path"] = file
    info["baseline"] = "new" if "new_baseline" in info["casename"] else "old"
    info["levels"] = int(path.parts[-4].split(".")[3].split("_")[-1][1:])

    # TODO: Use keith's util
    info["frequency"] = "daily" if xr.infer_freq(ds.time) == "D" else "monthly"

    info["variables"] = list(
        [
            key
            for key, var in ds.data_vars.items()
            if "time" in var.dims and "average" not in key
        ]
    )
    ds.close()
    return info


path = "/glade/campaign/cgd/oce/projects/pump/cesm/gmom.e23.GJRAv3.TL319_t061_zstar_N65.new_baseline.hb/run/jsons/h.json"
parse_cesm_mom6_kerchunk_json(str(path))

{'casename': 'gmom.e23.GJRAv3.TL319_t061_zstar_N65.new_baseline.hb',
 'stream': 'h',
 'path': '/glade/campaign/cgd/oce/projects/pump/cesm/gmom.e23.GJRAv3.TL319_t061_zstar_N65.new_baseline.hb/run/jsons/h.json',
 'baseline': 'new',
 'levels': 65,
 'frequency': 'monthly',
 'variables': ['N2_int',
  'agessc',
  'h',
  'rhopot0',
  'so',
  'thetao',
  'time_bnds',
  'uhGM',
  'uhml',
  'umo',
  'uo',
  'vhGM',
  'vhml',
  'vmo',
  'vo',
  'volcello']}

## Make catalog

In [ ]:
builder = Builder(paths=tuple(str(p) for p in ROOT.glob("**/run/jsons/")), depth=0)

In [ ]:
builder.build(parsing_func=parse_cesm_mom6_kerchunk_json)

Builder(paths=['/glade/campaign/cgd/oce/projects/pump/cesm/gmom.e23.GJRAv3.TL319_t061_zstar_N65.new_baseline.kpp.lmd.005.mixpods/run/jsons', '/glade/campaign/cgd/oce/projects/pump/cesm/gmom.e23.GJRAv3.TL319_t061_zstar_N65.baseline.kpp.lmd.004.mixpods/run/jsons', '/glade/campaign/cgd/oce/projects/pump/cesm/gmom.e23.GJRAv3.TL319_t061_zstar_N65.baseline.kpp.lmd.002.mixpods/run/jsons', '/glade/campaign/cgd/oce/projects/pump/cesm/gmom.e23.GJRAv3.TL319_t061_zstar_N150.kpp.lmd.004.mixpods/run/jsons', '/glade/campaign/cgd/oce/projects/pump/cesm/gmom.e23.GJRAv3.TL319_t061_zstar_N150.baseline.mixpods/run/jsons', '/glade/campaign/cgd/oce/projects/pump/cesm/gmom.e23.GJRAv3.TL319_t061_zstar_N65.new_baseline.kpp.lmd.004.mixpods/run/jsons', '/glade/campaign/cgd/oce/projects/pump/cesm/gmom.e23.GJRAv3.TL319_t061_zstar_N65.baseline.kpp.lmd.003.mixpods/run/jsons', '/glade/campaign/cgd/oce/projects/pump/cesm/gmom.e23.GJRAv3.TL319_t061_zstar_N65.baseline.epbl.001.mixpods/run/jsons', '/glade/campaign/cgd/oc

In [ ]:
builder.save(
    name="../pump-catalog",
    # Column name including filepath
    path_column_name="path",
    # Column name including variables
    variable_column_name="variables",
    # Data file format - could be netcdf or zarr (in this case, netcdf)
    data_format="reference",
    # Which attributes to groupby when reading in variables using intake-esm
    groupby_attrs=[],
    # Aggregations which are fed into xarray when reading in data using intake
    aggregations=[],
)

Successfully wrote ESM catalog json file to: file:///glade/u/home/dcherian/pump/notebooks/../pump-catalog.json


## Read Catalog

In [ ]:
import intake

data_catalog = intake.open_esm_datastore("../pump-catalog.json")
data_catalog

,unique
casename,10
stream,3
path,22
baseline,2
levels,2
frequency,2
variables,7
derived_variables,0


In [ ]:
data_catalog.df

,casename,stream,path,baseline,levels,frequency,variables
0,gmom.e23.GJRAv3.TL319_t061_zstar_N150.baseline...,sfc,/glade/campaign/cgd/oce/projects/pump/cesm/gmo...,old,150,daily,"['SSH', 'SSU', 'SSV', 'mlotst', 'oml', 'sos', ..."
1,gmom.e23.GJRAv3.TL319_t061_zstar_N150.kpp.lmd....,sfc,/glade/campaign/cgd/oce/projects/pump/cesm/gmo...,old,150,daily,"['SSH', 'SSU', 'SSV', 'mlotst', 'oml', 'sos', ..."
2,gmom.e23.GJRAv3.TL319_t061_zstar_N65.baseline....,h,/glade/campaign/cgd/oce/projects/pump/cesm/gmo...,old,65,monthly,"['N2_int', 'agessc', 'h', 'rhopot0', 'so', 'th..."
3,gmom.e23.GJRAv3.TL319_t061_zstar_N65.baseline....,sfc,/glade/campaign/cgd/oce/projects/pump/cesm/gmo...,old,65,monthly,"['Rd_dx', 'SSH', 'SSU', 'SSV', 'mass_wt', 'mlo..."
4,gmom.e23.GJRAv3.TL319_t061_zstar_N65.baseline....,h,/glade/campaign/cgd/oce/projects/pump/cesm/gmo...,old,65,monthly,"['h', 'so', 'thetao', 'time_bnds', 'uhGM', 'uh..."
5,gmom.e23.GJRAv3.TL319_t061_zstar_N65.baseline....,sfc,/glade/campaign/cgd/oce/projects/pump/cesm/gmo...,old,65,monthly,"['SSH', 'SSU', 'SSV', 'mlotst', 'sos', 'speed'..."
6,gmom.e23.GJRAv3.TL319_t061_zstar_N65.baseline....,wci,/glade/campaign/cgd/oce/projects/pump/cesm/gmo...,old,65,monthly,"['T_advection_xy', 'T_lbdxy_cont_tendency', 'T..."
7,gmom.e23.GJRAv3.TL319_t061_zstar_N65.baseline....,h,/glade/campaign/cgd/oce/projects/pump/cesm/gmo...,old,65,monthly,"['h', 'so', 'thetao', 'time_bnds', 'uhGM', 'uh..."
8,gmom.e23.GJRAv3.TL319_t061_zstar_N65.baseline....,sfc,/glade/campaign/cgd/oce/projects/pump/cesm/gmo...,old,65,monthly,"['SSH', 'SSU', 'SSV', 'mlotst', 'oml', 'sos', ..."
9,gmom.e23.GJRAv3.TL319_t061_zstar_N65.baseline....,wci,/glade/campaign/cgd/oce/projects/pump/cesm/gmo...,old,65,monthly,"['KPP_NLT_temp_budget', 'T_advection_xy', 'T_l..."


## Display interactive catalog

In [ ]:
from ipyaggrid import Grid
from IPython.display import HTML

df = data_catalog.df

column_defs = [
    {"headerName": "variable", "field": "variable", "rowGroup": False},
    {"headerName": "casename", "field": "casename", "rowGroup": False},
    {"headerName": "stream", "field": "stream", "rowGroup": False},
    {"headerName": "baseline", "field": "experiment", "rowGroup": False},
    {"headerName": "frequency", "field": "frequency", "rowGroup": False},
    {"headerName": "levels", "field": "levels", "rowGroup": False},
    {"headerName": "path", "field": "path", "rowGroup": False},
]

grid_options = {
    "columnDefs": column_defs,
    "enableSorting": True,
    "enableFilter": True,
    "enableColResize": True,
    "enableRangeSelection": True,
}

css_rules = """
.ag-row-hover {
    background-color: lightblue !important;
}

.ag-column-hover {
    background-color: powderblue;
}

.ag-row-hover .ag-column-hover {
    background-color: deepskyblue !important;
}
"""

g = Grid(
    grid_data=df,
    grid_options=grid_options,
    quick_filter=True,
    show_toggle_edit=False,
    export_csv=False,
    export_excel=False,
    theme="ag-theme-blue",
    show_toggle_delete=False,
    columns_fit="auto",
    index=False,
    keep_multiindex=False,
    css_rules=css_rules,
)

html = g.export_html(build=True)
HTML(html)